# Julia Design Patterns

Based on the talk [Dispatching Design Patterns](https://www.youtube.com/watch?v=n-E-1-A_rZM&t=956s) by Aaron Christianson

## Structs

* Immutable by default
* It is important to declare the types in a struct

In [11]:
struct Point
    x::Float64
    y::Float64
end

In [12]:
my_point = Point(5, 7)

Point(5.0, 7.0)

In [13]:
my_point.x 

5.0

In [14]:
my_point.y = 10

LoadError: setfield! immutable struct of type Point cannot be changed

### We can make a struct mutable

In [15]:
mutable struct Starship
    name::String
    location::Point
end

In [16]:
ship = Starship("SS1", Point(5, 6))

Starship("SS1", Point(5.0, 6.0))

In [19]:
ship.location = Point(1, 1)

Point(1.0, 1.0)

### Another way to make a struct

In [23]:
mutable struct Starship
    name::String
    location::Point
    Starship(name, x, y) = new(name, Point(x, y)) # Defining initializiation
end

## Methods

In [24]:
"""
Moving our ship
"""
function move!(starship, heading, distance)
    Δx = distance * cosd(heading)
    Δy = distance * sind(heading)
    old_location = starship.location
    
    starship.location = Point(old_location.x + Δx, old_location.y + Δy)
end

move! (generic function with 1 method)

In [25]:
ss1 = Starship("SS1", 3, 4)

Starship("SS1", Point(3.0, 4.0))

In [26]:
move!(ss1, 45, 1.5)

Point(4.060660171779821, 5.060660171779821)

As oposed to OOP, methods in julia are attached to functions and **not** to objects

In [27]:
## Defining two structs ##
struct Rectangle
    width::Float64
    height::Float64
end

struct Square
    length::Float64
end

In [28]:
## Defining methods for the structs ##

width(r::Rectangle) = r.width
height(r::Rectangle) = r.height

width(r::Square) = r.length
height(r::Square) = r.length

height (generic function with 2 methods)

In [29]:
rec1 = Rectangle(10, 40)
sq1 = Square(5)

Square(5.0)

In [31]:
width(rec1)

10.0

In [32]:
width(sq1)

5.0

In [33]:
area(shape) = width(shape) * height(shape)

area(rec1)

400.0

In [34]:
area(sq1)

25.0

## Polymorphism (Abstract types)

* Julia provides **abstract types** to make hierarchies of types with shared behaviour

In [42]:
# Abstract types do not have a data layout (a "superclass")
abstract type Shape end

In [43]:
# But we can define methods in terms of abstract types
combined_area(a::Shape, b::Shape) = area(a) + area(b)

combined_area (generic function with 1 method)

### Inheritance

* Next, we assign structs to an abstract types by *inheritance*

In [47]:
struct Circle <: Shape
    diameter::Float64
end

radius(c::Circle) = c.diameter / 2
area(c::Circle) = π * radius(c) ^ 2

area (generic function with 2 methods)

In [48]:
# Abstract types can also by subtypes of other abstract types
abstract type AbstractRectangle <: Shape end
area(r::AbstractRectangle) = width(r) * height(r)

area (generic function with 3 methods)

In [64]:
struct Rectangle2 <: AbstractRectangle
    width::Float64
    height::Float64
end
width(r::Rectangle2) = r.width
height(r::Rectangle2) = r.height


struct Square2 <: AbstractRectangle
    length::Float64
end
width(r::Square2) = r.length
height(r::Square2) = r.length

height (generic function with 4 methods)

In [60]:
rec2 = Rectangle2(3, 2)
sq2 = Square2(3)

Square2(3.0)

In [61]:
area(rec2)

6.0

In [65]:
area(sq2)

9.0

In [68]:
const c = Circle(3)
const s = Square2(3)
const r = Rectangle2(3, 2)

Rectangle2(3.0, 2.0)

In [69]:
combined_area(c, s)

16.068583470577035

In [74]:
@assert combined_area(s, r) == 15.0

## Parametric Types

In [82]:
# Here T is a type variable (not yet defined)
struct Point3{T}
    x::T
    y::T
end

Point3(1, 2) # Infer the type of object to Point3

Point3{Float64}(1.0, 2.0)

In [83]:
Point3(1.1, 2) # Error if T is not same for elements

LoadError: MethodError: no method matching Point3(::Float64, ::Int64)
Closest candidates are:
  Point3(::T, !Matched::T) where T at In[82]:3

In [89]:
# We can then define the type we want to use
Point3{Float64}(1, 3)

Point3{Float64}(1.0, 3.0)

We may want to add some form of constraint to our parametric types

In [90]:
Point3("hello", "world") # This should be invalid

Point3{String}("hello", "world")

In [91]:
# Constrained type variable in the hierarchy of reals
struct Point4{T <: Real}
    x::T
    y::T
end

In [92]:
Point4(3,4)

Point4{Int64}(3, 4)

In [94]:
Point4(3.,4.)

Point4{Float64}(3.0, 4.0)

In [95]:
Point4("foo", "bar")

LoadError: MethodError: no method matching Point4(::String, ::String)

## Custom container types

In [1]:
struct Nil end

struct List{T}
    head::T
    tail::Union{List{T}, Nil}
end

In [3]:
?foldr

search: foldr mapfoldr foldl mapfoldl



```
foldr(op, itr; [init])
```

Like [`reduce`](@ref), but with guaranteed right associativity. If provided, the keyword argument `init` will be used exactly once. In general, it will be necessary to provide `init` to work with empty collections.

# Examples

```jldoctest
julia> foldr(=>, 1:4)
1 => (2 => (3 => 4))

julia> foldr(=>, 1:4; init=0)
1 => (2 => (3 => (4 => 0)))
```
